In [1]:
import re
import pandas as pd
from collections import defaultdict

In [17]:
RUN_FILE = f'C:/Users/dell/Desktop/IWAN_shrun.txt'
#INT_STATUS_FILE = f'C:/Users/dell/Desktop/DR5K2_SG_SH_STAT.txt'
INT_STATUS_FILE = f'C:/Users/dell/Desktop/IWANintstat.txt'
CDP_FILE = f'C:/Users/dell/Desktop/IWANcdpdet.txt'
OUTPUT_FILE = f'C:/Users/dell/Desktop/switch_config.xlsx'

In [3]:
interfaces = defaultdict(lambda: {
    "Description": "",
    "Status": "",
    "VLANs": "",
    "IP Address": "",
    "Neighbour": "",
    "Neighbour Interface": ""
})

In [4]:
def normalize_interface_name(if_name):
    """
    Convert NX-OS or abbreviated interface names to full standard format.
    Examples:
        Eth1/1 -> Ethernet1/1
        Gi1/0/1 -> GigabitEthernet1/0/1
        Te1/1 -> TenGigabitEthernet1/1
        Po10 -> Port-channel10
    """
    if not if_name:
        return if_name

    if if_name.startswith("Eth"):
        return if_name.replace("Eth", "Ethernet", 1)
    elif if_name.startswith("Gi"):
        return if_name.replace("Gi", "GigabitEthernet", 1)
    elif if_name.startswith("Te"):
        return if_name.replace("Te", "TenGigabitEthernet", 1)
    elif if_name.startswith("Po"):
        return if_name.replace("Po", "Port-channel", 1)
    else:
        return if_name


In [33]:


# ================== PARSE SHOW RUN ==================
def parse_show_run(show_run_file):
    interfaces = {}
    current_interface = None

    with open(show_run_file, "r") as f:
        for line in f:
            line = line.strip()

            if line.startswith("interface"):
                current_interface = line.split()[1]
                interfaces[current_interface] = {
                    "Description": "",
                    "VLANs": "",
                    "IP Address": "",
                    "Port Channel": ""
                }

            elif current_interface:
                if line.startswith("description"):
                    interfaces[current_interface]["Description"] = line.replace("description", "").strip()

                elif line == "shutdown":
                    interfaces[current_interface]["Shutdown"] = "down"

                elif line.startswith("ip address"):
                    interfaces[current_interface]["IP Address"] = " ".join(line.split()[2:])

                elif "switchport access vlan" in line:
                    interfaces[current_interface]["VLANs"] = line.split()[-1]

                elif "switchport trunk allowed vlan" in line:
                    interfaces[current_interface]["VLANs"] = line.split()[-1]

                elif line.startswith("channel-group"):
                    channel_number = line.split()[1]
                    interfaces[current_interface]["Port Channel"] = f"Port-channel{channel_number}"

    return interfaces


# ================== PARSE SHOW INTERFACE STATUS ==================

def detect_columns(header_line):
    """
    Detect column names and their start positions from header line
    """
    columns = []
    for match in re.finditer(r"\S+", header_line):
        col_name = match.group()
        start = match.start()
        columns.append((col_name, start))
    return columns


def parse_ios_show_interface_status(lines):
    interfaces = {}

    header = lines[0]
    column_defs = detect_columns(header)

    # Build slice ranges
    slices = []
    for i in range(len(column_defs)):
        col_name, start = column_defs[i]
        end = column_defs[i + 1][1] if i + 1 < len(column_defs) else None
        slices.append((col_name, start, end))

    # Parse data rows
    for line in lines[1:]:
        if not line.strip():
            continue

        row = {}
        for col_name, start, end in slices:
            value = line[start:end].strip() if end else line[start:].strip()
            row[col_name] = value

        port = row.get("Port")
        if port:
            port = normalize_interface_name(port)

            interfaces[port] = row.get("Status", "")
           
            

    return interfaces

def read_txt_file(file_path):
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read().splitlines()
    

def parse_show_interface_status(int_status_file): 
    lines = read_txt_file(int_status_file)
    int_status = parse_ios_show_interface_status(lines)
    return int_status



# ================== PARSE SHOW CDP NEIGHBORS ==================


def parse_ios_cdp_neighbors_detail(show_cdp_file):
    """
    Parse IOS 'show cdp neighbors detail' output.
    Returns:
        {
            local_interface: {
                'Neighbour': device_id,
                'Neighbour Interface': neighbor_port
            }
        }
    """

    cdp_neighbors = {}

    current_device = None
    local_interface = None
    neighbor_interface = None

    with open(show_cdp_file, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()

            if not line:
                continue

            # Device ID
            if line.startswith("Device ID:"):
                current_device = line.split(":", 1)[1].strip()
                continue

            # Interface line
            if line.startswith("Interface:") and "Port ID" in line:
                # Example:
                # Interface: GigabitEthernet0/1,  Port ID (outgoing port): GigabitEthernet1/0/24
                try:
                    left, right = line.split(",  Port ID (outgoing port):", 1)

                    local_interface =left.replace("Interface:", "").strip()
                
                    neighbor_interface = right.strip()

                    if current_device and local_interface:
                        cdp_neighbors[local_interface] = {
                            "Neighbour": current_device,
                            "Neighbour Interface": neighbor_interface
                        }

                except ValueError:
                    continue

    return cdp_neighbors




# ================== MAIN ==================
def main():
    show_run_data = parse_show_run(RUN_FILE)
    interface_status_data = parse_show_interface_status(INT_STATUS_FILE)
    cdp_neighbor_data = parse_ios_cdp_neighbors_detail(CDP_FILE)

    rows = []

    for interface, data in show_run_data.items():
        row = {
            "Interface": interface,
            "Description": data["Description"],
            "Status": interface_status_data.get(interface),
            "VLANs": data["VLANs"],
            "IP Address": data["IP Address"],
            "Port Channel": data["Port Channel"],
            "Neighbour": cdp_neighbor_data.get(interface, {}).get("Neighbour", ""),
            "Neighbour Interface": cdp_neighbor_data.get(interface, {}).get("Neighbour Interface", "")
        }
        rows.append(row)

    df = pd.DataFrame(rows)
    df.to_excel(OUTPUT_FILE, index=False)

    print(f"Excel file successfully created:\n{OUTPUT_FILE}")


if __name__ == "__main__":
    main()


Excel file successfully created:
C:/Users/dell/Desktop/switch_config.xlsx


In [27]:

def detect_columns(header_line):
    """
    Detect column names and their start positions from header line
    """
    columns = []
    for match in re.finditer(r"\S+", header_line):
        col_name = match.group()
        start = match.start()
        columns.append((col_name, start))
    return columns


def parse_nxos_show_interface_status(lines):
    interfaces = {}

    header = lines[0]
    column_defs = detect_columns(header)

    # Build slice ranges
    slices = []
    for i in range(len(column_defs)):
        col_name, start = column_defs[i]
        end = column_defs[i + 1][1] if i + 1 < len(column_defs) else None
        slices.append((col_name, start, end))

    # Parse data rows
    for line in lines[1:]:
        if not line.strip():
            continue

        row = {}
        for col_name, start, end in slices:
            value = line[start:end].strip() if end else line[start:].strip()
            row[col_name] = value

        port = row.get("Port")
        if port:
            port = normalize_interface_name(port)

            interfaces[port] = row.get("Status", "")
           
            

    return interfaces


In [ ]:
def read_txt_file(file_path):
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read().splitlines()
    


In [29]:
lines = read_txt_file(INT_STATUS_FILE)

int_status = parse_nxos_show_interface_status(lines)



int_status



{'GigabitEthernet1/0/1': 'connected',
 'GigabitEthernet1/0/2': 'notconnect',
 'GigabitEthernet1/0/3': 'notconnect',
 'GigabitEthernet1/0/4': 'connected',
 'GigabitEthernet1/0/5': 'notconnect',
 'GigabitEthernet1/0/6': 'connected',
 'GigabitEthernet1/0/7': 'notconnect',
 'GigabitEthernet1/0/8': 'notconnect',
 'GigabitEthernet1/0/9': 'notconnect',
 'GigabitEthernet1/0/10': 'notconnect',
 'GigabitEthernet1/0/11': 'notconnect',
 'GigabitEthernet1/0/12': 'connected',
 'GigabitEthernet1/0/13': 'notconnect',
 'GigabitEthernet1/0/14': 'notconnect',
 'GigabitEthernet1/0/15': 'notconnect',
 'GigabitEthernet1/0/16': 'notconnect',
 'GigabitEthernet1/0/17': 'notconnect',
 'GigabitEthernet1/0/18': 'notconnect',
 'GigabitEthernet1/0/19': 'notconnect',
 'GigabitEthernet1/0/20': 'connected',
 'GigabitEthernet1/0/21': 'connected',
 'GigabitEthernet1/0/22': 'connected',
 'GigabitEthernet1/0/23': 'connected',
 'GigabitEthernet1/0/24': 'connected',
 'GigabitEthernet1/0/25': 'notconnect',
 'GigabitEthernet1

In [12]:
def parse_show_interface_status(ios_status_file):
    """
    Parse NX-OS 'show interface status' fixed-width output into a dictionary:
    { interface_name: operational_status }
    """
    interface_status = {}

    with open(ios_status_file, "r") as f:
        for line in f:
            # Skip empty lines or headers/separators
            if not line.strip() or line.startswith('Port') or line.startswith('--'):
                continue

            # NX-OS fixed column positions (example typical widths):
            # Port(13), Name(19), Status(13), Vlan(10), Duplex(7), Speed(9), Type(20)
            interface_name = line[0:13].strip()
            status = line[32:43].strip()  # Status column

             # Normalize interface names
            interface_name = normalize_interface_name(interface_name)

            # Only include valid NX-OS states
            valid_states = ['connected', 'disabled', 'errDisabled']
            if status in valid_states:
                interface_status[interface_name] = status

    return interface_status

In [16]:
a = parse_show_interface_status(INT_STATUS_FILE)
a

{'Ethernet1/5': 'connected',
 'Ethernet1/6': 'connected',
 'Ethernet1/7': 'connected',
 'Ethernet1/8': 'connected',
 'Ethernet1/9': 'connected',
 'Ethernet1/10': 'connected',
 'Ethernet1/11': 'connected',
 'Ethernet1/12': 'connected',
 'Ethernet1/13': 'connected',
 'Ethernet1/14': 'connected',
 'Ethernet1/15': 'connected',
 'Ethernet1/16': 'connected',
 'Ethernet1/17': 'connected',
 'Ethernet1/18': 'connected',
 'Ethernet1/19': 'connected',
 'Ethernet1/20': 'connected',
 'Ethernet1/21': 'connected',
 'Ethernet1/22': 'connected',
 'Ethernet1/23': 'connected',
 'Ethernet1/24': 'connected',
 'Ethernet1/25': 'connected',
 'Ethernet1/26': 'connected',
 'Ethernet1/27': 'connected',
 'Ethernet1/28': 'connected',
 'Port-channel53': 'connected',
 'Port-channel54': 'connected',
 'Port-channel55': 'connected',
 'Port-channel56': 'connected',
 'Port-channel57': 'connected',
 'Port-channel58': 'connected',
 'Port-channel59': 'connected',
 'Port-channel61': 'connected',
 'Port-channel79': 'connected